In [1]:
from __future__ import print_function
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## init stereo aruco detector scene builder

In [2]:
from pkg.marker_config import *
from pkg.detector.aruco.stereo import ArucoStereo
from pkg.detector.camera.realsense import RealSense
from pkg.detector.camera.kinect import Kinect
from pkg.geometry.builder.scene_builder import SceneBuilder

s_builder = SceneBuilder(None, base_link="base_link")
# s_builder.reset_reference_coord(ref_name="floor")

In [3]:
from pkg.planning.filtering.reach_filter import ReachTrainer
from pkg.controller.combined_robot import *

In [4]:
rtrain = ReachTrainer(scene_builder=s_builder)

## collect and train

In [ ]:
rtrain.collect_and_learn(RobotType.indy7, "indy0_tcp", 
                         TRAIN_COUNT=20000, TEST_COUNT=5000, save_data=True, save_model=True, timeout=0.3)
import matplotlib.pyplot as plt
plt.plot(rtrain.time_plan, '.')

connection_list
[False]


Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


added robot collision boundaries
0 / 20000 (0 / 6673159 s): current success ratio = 0.0
100 / 20000 (26 / 5212 s): current success ratio = 0.237623762376


In [ ]:
rtrain.collect_and_learn(RobotType.panda, "panda0_hand", 
                         TRAIN_COUNT=20000, TEST_COUNT=5000, save_data=True, save_model=True, timeout=1)
import matplotlib.pyplot as plt
plt.plot(rtrain.time_plan, '.')

## load and train

In [5]:
rtrain.load_and_learn(RobotType.indy7)
rtrain.load_and_learn(RobotType.panda)

/home/rnb/.local/lib/python2.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


trainset: 	317.0 ms/1 = 316.597 ms (316.597/316.597)
testset: 	78.0 ms/1 = 78.225 ms (78.225/78.225)

trainning accuracy = 99.44 %
test accuracy = 98.44 %
trainning success accuracy = 99.38 %
trainning failure accuracy = 99.46 %
test success accuracy = 98.02 %
test failure accuracy = 98.59 %
trainset: 	390.0 ms/1 = 390.258 ms (390.258/390.258)
testset: 	97.0 ms/1 = 97.452 ms (97.452/97.452)

trainning accuracy = 98.9 %
test accuracy = 98.16 %
trainning success accuracy = 97.94 %
trainning failure accuracy = 99.11 %
test success accuracy = 95.29 %
test failure accuracy = 98.77 %


## load and test

In [6]:
rtrain.load_and_test(RobotType.indy7)
rtrain.load_and_test(RobotType.panda)

trainset: 	318.0 ms/1 = 318.007 ms (318.007/318.007)
testset: 	79.0 ms/1 = 79.133 ms (79.133/79.133)

trainning accuracy = 99.44 %
test accuracy = 98.44 %
trainning success accuracy = 99.38 %
trainning failure accuracy = 99.46 %
test success accuracy = 98.02 % (1286/1312)
test failure accuracy = 98.59 % (3636/3688)
trainset: 	392.0 ms/1 = 392.411 ms (392.411/392.411)
testset: 	98.0 ms/1 = 98.381 ms (98.381/98.381)

trainning accuracy = 98.9 %
test accuracy = 98.16 %
trainning success accuracy = 97.94 %
trainning failure accuracy = 99.11 %
test success accuracy = 95.29 % (829/870)
test failure accuracy = 98.77 % (4079/4130)


In [ ]:
np.where(self.success_list_test)